In [1]:
import ads
import pandas as pd
ads.config.token='42guhMWZUbJFFB5XPzhLJ14LScoO240jZvlkzPk3'

In [2]:
papers = ads.SearchQuery(q='(((full:"tardis" AD (full:"kerzendorf" OR (bibstem:"Natur" AND full:"supernova")))) AND year:2014-)+property:refereed', sort="date",
                         fl = ['bibcode','title','first_author','alternate_bibcode'])

bibcodes = []
titles = []
first_authors = []
arxiv_codes = []
for paper in papers:
    bibcodes.append(paper.bibcode)
    titles.append(paper.title)
    first_authors.append(paper.first_author)
    arxiv_codes.append(paper.alternate_bibcode) #

In [3]:
def format_arxiv_codes(arxiv_codes):
    """This takes what is returned by paper.alternate_bibcode and formats it into a searchable arXiv code.
    What is default returned is YEAR(4digits)arXiv######## but we need arXiv####:####~"""
    arxiv_codes_no_year = []
    for item in arxiv_codes:
        try:
            if len(item) == 1:
                no_year = item[0][4:]
                arxiv_codes_no_year.append(no_year)
            else:
                if item[0].find('arXiv') == -1: #If the find statement cannot find it, it returns -1
                    arxiv_codes_no_year.append(item[1][4:])
                else:
                    arxiv_codes_no_year.append(item[0][4:])
        except TypeError: #This is the error you get if you try to do the len(None)
            arxiv_codes_no_year.append('no arxiv') 
            

    arxiv_codes_proper_format = []
    for initial_item in arxiv_codes_no_year:
        if initial_item != "no arxiv":
            temp_list = []
            for item in initial_item:
                temp_list.append(item)
        
            if temp_list[5] != ":":
                temp_list.insert(5,":")
            if temp_list[10] != ".":
                temp_list.insert(10, ".")

            good_str = ""
            for item in temp_list:
                good_str = good_str + item
            arxiv_codes_proper_format.append(good_str)
        elif initial_item == "no arxiv": 
            arxiv_codes_proper_format.append("no arxiv")
            
    return arxiv_codes_proper_format

In [4]:
d = {'bibcode': bibcodes, 'title': titles, 'first author': first_authors, 'arxiv': format_arxiv_codes(arxiv_codes)}
df = pd.DataFrame(data=d)
df.to_csv('adslist.csv')

In [5]:

combo_list = zip(bibcodes, arxiv_codes)
for item in combo_list:
    print(item) 

('2020ApJS..250...12C', ['2019arXiv191105209C'])
('2020ApJ...898...56M', ['2020arXiv200505972M'])
('2020MNRAS.497..246G', ['2020arXiv200712110G'])
('2020ApJS..249....8B', None)
('2020MNRAS.496.1132T', ['2020arXiv200200393T', '2020MNRAS.tmp.1787T'])
('2020MNRAS.494.5811L', ['2019arXiv191204313L', '2020MNRAS.tmp.1131L'])
('2020ApJ...893..143K', ['2019arXiv190803001K'])
('2020ApJ...892L..24S', ['2020arXiv200109722S'])
('2020A&A...634A..37M', ['2019arXiv191207603M'])
('2020A&A...633A..88V', ['2019arXiv191104444V'])
('2019ApJ...885L..23M', ['2019arXiv190904545M'])
('2019Natur.574..497W', ['2019arXiv191010510W'])
('2019MNRAS.487.2538J', ['2018arXiv181211692J', '2019MNRAS.tmp.1251J'])
('2019LRCA....5....1N', ['2019arXiv190709840N'])
('2019ApJ...876..148C', ['2017arXiv171210091C'])
('2019MNRAS.484.4785M', ['2019MNRAS.tmp..239M', '2019arXiv190108582M'])
('2019A&A...622A.102M', ['2018arXiv181208695M'])
('2019ApJ...871..250H', ['2019arXiv190201904H'])
('2019Natur.565..324I', ['2019arXiv190105500I

The .alternate_bibcode statement adds the year, and it removes the formatting of the arXiv code. For search on the website, the input needs to be arXiv:####.stuff after. The function format_arxiv_codes creates the proper format for the codes to be used in search.

In [6]:
comparison = zip(arxiv_codes, format_arxiv_codes(arxiv_codes))
for item in comparison:
    print(item)

(['2019arXiv191105209C'], 'arXiv:1911.05209C')
(['2020arXiv200505972M'], 'arXiv:2005.05972M')
(['2020arXiv200712110G'], 'arXiv:2007.12110G')
(None, 'no arxiv')
(['2020arXiv200200393T', '2020MNRAS.tmp.1787T'], 'arXiv:2002.00393T')
(['2019arXiv191204313L', '2020MNRAS.tmp.1131L'], 'arXiv:1912.04313L')
(['2019arXiv190803001K'], 'arXiv:1908.03001K')
(['2020arXiv200109722S'], 'arXiv:2001.09722S')
(['2019arXiv191207603M'], 'arXiv:1912.07603M')
(['2019arXiv191104444V'], 'arXiv:1911.04444V')
(['2019arXiv190904545M'], 'arXiv:1909.04545M')
(['2019arXiv191010510W'], 'arXiv:1910.10510W')
(['2018arXiv181211692J', '2019MNRAS.tmp.1251J'], 'arXiv:1812.11692J')
(['2019arXiv190709840N'], 'arXiv:1907.09840N')
(['2017arXiv171210091C'], 'arXiv:1712.10091C')
(['2019MNRAS.tmp..239M', '2019arXiv190108582M'], 'arXiv:1901.08582M')
(['2018arXiv181208695M'], 'arXiv:1812.08695M')
(['2019arXiv190201904H'], 'arXiv:1902.01904H')
(['2019arXiv190105500I'], 'arXiv:1901.05500I')
(['2018arXiv181102543V'], 'arXiv:1811.02543